In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# Memory

A maioria das aplicações de Modelos de LLM possui uma interface conversacional. Um componente essencial de uma conversa é a capacidade de se referir a informações introduzidas anteriormente na conversa. No mínimo, um sistema conversacional deve ser capaz de acessar diretamente alguma janela de mensagens passadas. Um sistema mais complexo precisará ter um modelo de mundo que está constantemente atualizando, o que lhe permite fazer coisas como manter informações sobre entidades e suas relações.

Chamamos essa capacidade de armazenar informações sobre interações passadas de "Memory", ou memória. LangChain oferece muitas utilidades para adicionar memória a um sistema. Essas utilidades podem ser usadas por si só ou incorporadas de maneira integrada em uma chain.

In [2]:
from langchain_core.chat_history import InMemoryChatMessageHistory

memory = InMemoryChatMessageHistory()

In [3]:
memory.add_user_message('Olá modelo')
memory.add_ai_message('Olá user')
memory.messages

[HumanMessage(content='Olá modelo', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Olá user', additional_kwargs={}, response_metadata={})]

## Criando uma conversa com memória

In [4]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate as cpt

prompt = cpt.from_messages([
    ('system', 'Você é um tutor de programação chamado Eduardo. Responda as perguntas de forma didática.'),
    ('placeholder', '{history}'),
    ('human', '{pergunta}')
])
chain = prompt | ChatOpenAI(model='gpt-4o-mini')

In [5]:
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chain_com_memoria = RunnableWithMessageHistory(
    chain,
    get_by_session_id,
    input_messages_key='pergunta',
    history_messages_key='history'
)

In [8]:
config = {'configurable': {'session_id': 'usuario_a'}}
resposta = chain_com_memoria.invoke({'pergunta': 'O meu nome é Thomas'}, config=config)
resposta.content

'Oi, Thomas! Entendi que você está se apresentando. Se precisar de ajuda com algo específico ou se tiver alguma pergunta, fique à vontade para compartilhar! Estou aqui para ajudar.'

In [9]:
resposta = chain_com_memoria.invoke({'pergunta': 'Qual é o meu nome?'}, config=config)
resposta.content

'Seu nome é Thomas! Se precisar de ajuda ou tiver perguntas, é só me avisar!'

In [10]:
config = {'configurable': {'session_id': 'usuario_b'}}
resposta = chain_com_memoria.invoke({'pergunta': 'Qual é o meu nome?'}, config=config)
resposta.content

'Desculpe, mas eu não tenho acesso a informações pessoais, como o seu nome. Posso te ajudar com perguntas sobre programação ou qualquer outro assunto que você precise! O que você gostaria de saber?'